In [1]:
import numpy as np
import tifffile
import os
import pickle
import joblib
import pandas as pd
from matplotlib import pyplot as plt
import concurrent.futures
import gc
import json
import sys

dt_annotated = np.dtype([
    ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
    ('atlas_id', 'u2'), ('is_positive2','bool'),('is_positive3','bool')
])

dt_stitched = np.dtype([
    ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
])

dt_classified = np.dtype([
    #('is_positive','bool'), ('is_positive2','bool'),('is_positive3','bool')
    ('is_positive','bool'), ('intensity','f4'),('intensity_2','f4'), ('intensity_3','f4')
])

def load_points(stack_dir, file_name):
    with open(os.path.join(stack_dir, file_name), "rb") as f:
        tmp = joblib.load(f)
        point = np.zeros(tmp.shape, dtype=dt_annotated)
        point[:] = tmp[:]           
        # numpy配列の各要素をPythonリストに変換
        list_array = point.tolist()
        # Pythonリストの各要素をnumpy配列に変換
        new_array = np.array([np.array(sublist) for sublist in list_array])
        del tmp,point,list_array
        gc.collect()
        return  new_array
    
def load_points_stitched(stack_dir, file_name):
    with open(os.path.join(stack_dir, file_name), "rb") as f:
        tmp = joblib.load(f)
        point = np.zeros(tmp.shape, dtype=dt_stitched)
        point[:] = tmp[:]           
        # numpy配列の各要素をPythonリストに変換
        list_array = point.tolist()
        # Pythonリストの各要素をnumpy配列に変換
        new_array = np.array([np.array(sublist) for sublist in list_array])
        del tmp,point,list_array
        gc.collect()
        return  new_array
    
def load_points_classified(stack_dir, file_name):
    with open(os.path.join(stack_dir, file_name), "rb") as f:
        tmp = joblib.load(f)
        point = np.zeros(tmp.shape, dtype=dt_classified)
        point[:] = tmp[:]           
        # numpy配列の各要素をPythonリストに変換
        list_array = point.tolist()
        # Pythonリストの各要素をnumpy配列に変換
        new_array = np.array([np.array(sublist) for sublist in list_array])
        del tmp,point,list_array
        gc.collect()
        return  new_array
    
def process_file (fw_dir_nu_, file_name):
    if file_name.endswith(".pkl"):
        #print(file_name)
        tmp_points_nu = load_points(fw_dir_nu_, file_name)
        try:
            tmp_points_classified = load_points_classified(fw_dir_nu_.replace('nu_R_/result', 'classified_'), file_name)            
            if tmp_points_classified.size > 0:
                tmp_points_nu = tmp_points_nu[tmp_points_classified.T[0]>0]

                del tmp_points_classified
                gc.collect()
                return tmp_points_nu
            else:
                return np.empty((3, 0))
            
        except Exception as e:
                return np.empty((3, 0))
    
def process_file_stitched (fw_dir_nu_, file_name):
    print(file_name)
    # classifyもしないといけないか。
    tmp_points_nu = np.fromfile(os.path.join(fw_dir_nu_, file_name), dtype=dt_stitched)
    try:
        tmp_points_classified = np.fromfile(os.path.join(fw_dir_nu_.replace('stitching_/stitched', 'classified_'), file_name), dtype=dt_classified)
        if tmp_points_classified.size > 0:
            tmp_points_nu = tmp_points_nu[tmp_points_classified["is_positive"]>0]
            #print(tmp_points_nu.shape)
            del tmp_points_classified
            gc.collect()
            return tmp_points_nu
        else:
            print("No classified file")
            #return np.empty((0, ), dtype=dt_stitched)
        
    except Exception as e:
        print("emp")
            #return np.empty((0, ), dtype=dt_stitched)
        
            

In [ ]:
conditions = {
    "WT_1m": ["#4_APPmodel_Ctr1m_1_2022_1104_1550","#4_APPmodel_Ctr1m_2_2022_1108_959","#4_APPmodel_Ctr1m_3_2022_1209_1751"],
    "WT_2m": ["8w_B6JWT_atlas_1_2022_0924_1847","8w_B6JWT_atlas_2_2022_0926_1326", "8w_B6JWT_atlas_3_2022_1005_1110", "8w_B6JWT_atlas_4_2022_1010_2030","8w_B6JWT_atlas_5_2023_0423_1604",],
    "WT_3m": ["#5_APPmodel_Ctr3m_1_2023_0113_1230", "#5_APPmodel_Ctr3m_2_2023_0210_1142", "#5_APPmodel_Ctr3m_3_2023_0227_1605", "#4_APPmodel_Ctr3m_4_2023_0427_1444",],
    "WT_5m": ["#4_APPmodel_Ctr5m_1_2022_1026_252", "#4_APPmodel_Ctr5m_2_2022_1028_2336", "#4_APPmodel_Ctr5m_3_2022_1101_140", "#4_APPmodel_Ctr5m_4_2023_0512_2057"],
    "WT_7m": [ "#5_APPmodel_Ctr7m_1_2023_0206_1534","#5_APPmodel_Ctr7m_2_2023_0313_1111","#5_APPmodel_Ctr7m_3_2023_0331_2139","#4_APPmodel_Ctr7m_4_2023_0516_1559","#5_APPmodel_Ctr7m_5_2023_0522_2218"],
    "WT_9m": [ "#5_APPmodel_Ctr9m_1_2023_0130_1607", "#5_APPmodel_Ctr9m_2_2023_0216_2052" ,"#5_APPmodel_Ctr9m_3_2023_0322_2014","#5_APPmodel_Ctr9m_4_2023_0518_1449"],
    "WT_12m": ["56w_B6JWT_atlas_1_2022_1128_1548", "56w_B6JWT_atlas_2_2022_1221_1537", "56w_B6JWT_atlas_3_2022_1224_1718"],
    
    "APP_1m": ["#4_APPmodel_APP1m_1_2022_1102_1304", "#4_APPmodel_APP1m_2_2022_1106_1708","#4_APPmodel_APP1m_3_2022_1213_2151","#4_APPmodel_APP1m_4_2023_0417_2052", ],
    "APP_3m": [ "#5_APPmodel_APP3m_2_2023_0123_1645", "#5_APPmodel_APP3m_3_2023_0212_2217","#5_APPmodel_APP3m_4_2023_0214_2027","#4_APPmodel_APP3m_5_2023_0502_1423",],
   "APP_5m": ["#4_APPmodel_APP5m_1_2022_1024_1336", "#4_APPmodel_APP5m_2_2022_1027_1333","#4_APPmodel_APP5m_3_2022_1030_1257", "#4_APPmodel_APP5m_4_2023_0509_1021"],
    "APP_7m": ["#5_APPmodel_APP7m_2_2023_0315_1540","#5_APPmodel_APP7m_3_2023_0310_1401","#5_APPmodel_APP7m_4_2023_0317_1630", "#5_APPmodel_APP7m_5_2023_0520_1723"],
    "APP_9m": ["#5_APPmodel_APP9m_1_2023_0117_1352","#5_APPmodel_APP9m_2_2023_0327_1425", "#5_APPmodel_APP9m_3_2023_0329_1611", "#5_APPmodel_APP9m_4_2023_0403_1443" ],
       
    "VCP_mut_2m": ["VCPmodel_E524A_0.5x_1_2022_0624_1303", "VCPmodel_E524A_0.5x_2_2022_0626_1338", "VCPmodel_E524A_0.5x_4_2022_0922_1217", "VCPmodel_E524A_0.5x_6_2022_0628_1157"], 
   "VCP_WT_2m": ["VCPmodel_WT_0.5x_1_2022_0625_1104","VCPmodel_WT_0.5x_2_2022_0917_1808", "VCPmodel_WT_0.5x_3_2022_0627_1257", "VCPmodel_WT_0.5x_4_2022_0921_950","VCPmodel_WT_0.5x_5_2022_0923_1454","VCPmodel_WT_0.5x_6_2022_0929_953"],
    "newVCP_mut_2m": ["#5_VCPmodel_E524A_0.5x_1_2023_0120_1929","#5_VCPmodel_E524A_0.5x_2_2023_0407_1031", "#5_VCPmodel_E524A_0.5x_3_2023_0413_2036"], 
   "newVCP_WT_2m": ["#5_VCPmodel_WT_0.5x_1_2023_0405_1108","#5_VCPmodel_WT_0.5x_2_2023_0115_1852","#5_VCPmodel_WT_0.5x_3_2023_0410_1246"], 
    
    "TMT_2m": ["#3_TMTmodel_TMT3.0_8w_1_2022_1014_1410", "#3_TMTmodel_TMT3.0_8w_2_2022_1020_938", "#3_TMTmodel_TMT3.0_8w_3_2022_1023_501", "#3_TMTmodel_TMT_8w_5_2022_1205_1147",],
  "Saline_2m": ["#3_TMTmodel_Saline_8w_1_2022_1016_1318", "#3_TMTmodel_Saline_8w_2_2022_1018_2103","#3_TMTmodel_Saline_8w_3_2022_1021_1910", "#3_TMTmodel_Saline_8w_4_2022_1202_1349", "#3_TMTmodel_Saline_8w_5_2022_1217_1723", "#3_TMTmodel_Saline_8w_6_2022_1216_001"],
    
}

In [4]:
Samples = [ "Neuronomics/" + path for paths in conditions.values() for path in paths]
Samples

In [3]:
for Sample in Samples:
    
    param_path = "/home/mitani/CUBIC-informatics/param/" + Sample + "/param_stitching.json"

    with open(param_path) as f:
        param = json.load(f)
    
    fw_dir_nu = param["dst_basedir"]+"stitched/"
    print(fw_dir_nu)

    points_nu = []
    tmp_points_nu = []
    tmp = []
    
    print("FW reading")
    fw_dir_nu_ = fw_dir_nu + "FW/"
    with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
        points_nu = list(executor.map(process_file_stitched, [fw_dir_nu_]*len(os.listdir(fw_dir_nu_)), os.listdir(fw_dir_nu_)))
    
    print("RV reading")
    fw_dir_nu_ = fw_dir_nu + "RV/"
    with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
        points_nu +=list(executor.map(process_file_stitched, [fw_dir_nu_]*len(os.listdir(fw_dir_nu_)), os.listdir(fw_dir_nu_)))
    
    print("image save")
    scale = 1/50

    # Nuclear
    ptsw2 = []
    points_nu_filtered = [arr for arr in points_nu if arr is not None and arr.ndim == 1]


    ptsw2 = np.concatenate(points_nu_filtered, axis=0)


    del points_nu_filtered
    gc.collect()

    depth_ori = int(np.floor(np.max(ptsw2['stitched_z']*scale)))+10
    height_ori =int(np.floor(np.max(ptsw2['stitched_y']*scale)))+10
    width_ori = int(np.floor(np.max(ptsw2['stitched_x']*scale)))+10

    img_filename_Nuclear_Isocortex_to_ori = param["dst_basedir"].replace("stitching","scalemerged") + "/img_density.tif"

    img_N_ori,_ = np.histogramdd(
        np.vstack([
            ptsw2['stitched_z']*scale,
            ptsw2['stitched_y']*scale,
            ptsw2['stitched_x']*scale
            ]).T,
            bins=(depth_ori, height_ori, width_ori),
            range=[(0,depth_ori),(0,height_ori),(0,width_ori)] # 240321 修正した。
        )

    tifffile.imsave(
            img_filename_Nuclear_Isocortex_to_ori,
            img_N_ori.astype(np.uint16)
        )
    print("Nuclear image save done")

    del img_N_ori
    gc.collect()
    

/export3/Imaging/Axial/Neurorology/#5_APPmodel_Ctr3m_3_2023_0227_1605/stitching_/stitched/
FW reading
242300_103400.bin148700_138500.bin

113600_138500.bin
078500_150200.bin
242300_185300.bin
090200_150200.bin218900_173600.bin230600_185300.bin207200_161900.bin


137000_138500.bin

183800_150200.bin125300_138500.bin055100_173600.bin


066800_161900.bin
101900_150200.bin
172100_150200.bin
195500_161900.bin
218900_185300.bin
207200_173600.bin
090200_161900.bin
160400_150200.bin
078500_161900.bin
254000_197000.bin
242300_197000.bin
113600_150200.bin
066800_173600.bin
183800_161900.bin
230600_197000.bin
148700_150200.bin
101900_161900.bin
055100_185300.bin
090200_173600.bin
078500_173600.bin
137000_150200.bin
172100_161900.bin
195500_173600.bin
125300_150200.bin
207200_185300.bin
160400_161900.bin
183800_173600.bin
020000_197000.bin
254000_208700.bin
043400_197000.bin
195500_185300.bin
101900_173600.bin
148700_161900.bin
242300_208700.bin
172100_173600.bin
218900_197000.bin
113600_161900.bi